In [ ]:
# VOEG BIJ EXCTRACT OVERAL 1577836800 TOE

In [8]:
import pandas as pd
import sys 
import os
#sys.path.append(os.path.abspath("C:\\Users\\lexfo\\Documents\\studeren\\Master_JADS\\thesis\\Code_for_BOT\\current_container\\Python_financial_container\\thesis_code"))
from LJT_database.make_query import *
from LJT_helper_functions.helpers import *

list_not_to_merge = ['econ_bitcoin', 'events_crypto']


In [9]:
def rename_columns(dataframe, coinpair, table):
    columns = list(dataframe.columns)
    new_columns = ['last_start_time']
    new_columns.extend(f"{coinpair}__{table}__{column}" for column in columns[1:])
    new_names = {columns[j]: new_columns[j] for j in range(len(columns))}
    return dataframe.rename(columns=new_names)

In [10]:
def merge_all_tables_in_dataset(coinpair, dict_with_structure):
    if coinpair in dict_with_structure.keys():
        all_tables_in_coinpair_dataset = dict_with_structure[coinpair]
        final_dataframe_coinpair = extract_dataset(coinpair, all_tables_in_coinpair_dataset[0],1581012000000.0) #create a dataframe from the first table in coinpair dataset to further merge with other tables
        final_dataframe_coinpair_unique = final_dataframe_coinpair.drop_duplicates(['last_start_time'])
        final_dataframe_coinpair_unique = rename_columns(final_dataframe_coinpair_unique, coinpair,all_tables_in_coinpair_dataset[0])

        for i in range(1, len(all_tables_in_coinpair_dataset)):
            print(all_tables_in_coinpair_dataset[i])
            dataframe = extract_dataset(coinpair, all_tables_in_coinpair_dataset[i],1581012000000.0)
            dataframe_unique = dataframe.drop_duplicates(['last_start_time'])
            dataframe_unique = rename_columns(dataframe_unique, coinpair, all_tables_in_coinpair_dataset[i])
            final_dataframe_coinpair_unique = pd.merge(final_dataframe_coinpair_unique, dataframe_unique, on='last_start_time',how='outer')
        print(' ---> Done', dict_with_structure[coinpair], 'are merged')

        final_dataframe_unique = final_dataframe_coinpair_unique.drop_duplicates(['last_start_time'])
        return final_dataframe_unique
    else:
        print('the coinpair cannot be found in Google BQ')

In [11]:
def merge_split_coins(coinpair, dict_with_structure):
    key, target = split_all_coinpairs(coinpair)
    print('Now', key.upper(), 'and', target.upper(), 'are merged')

    if len(dict_with_structure[key.upper()]) == 1 and len(dict_with_structure[target.upper()]) == 1:
        dataframe_of_key = extract_dataset(key.upper(), dict_with_structure[key.upper()][0],1581012000000.0)
        dataframe_of_key_unique = dataframe_of_key.drop_duplicates(['last_start_time'])
        dataframe_of_key_unique = rename_columns(dataframe_of_key_unique, key.upper(),dict_with_structure[key.upper()][0])
        dataframe_of_target = extract_dataset(target.upper(), dict_with_structure[target.upper()][0],1581012000000.0)
        dataframe_of_target_unique = dataframe_of_target.drop_duplicates(['last_start_time'])
        dataframe_of_target_unique = rename_columns(dataframe_of_target_unique, target.upper(),dict_with_structure[target.upper()][0])

        final_dataset_individual_coins = pd.merge(dataframe_of_key_unique, dataframe_of_target_unique,on='last_start_time',how='outer')
        print('---> Done', key.upper(), target.upper(), 'are merged')
        final_dataset_individual_coins_unique = final_dataset_individual_coins.drop_duplicates(['last_start_time'])
        return final_dataset_individual_coins_unique
    else:
        print('.... Merging tables of', key.upper())
        final_dataset_key = merge_all_tables_in_dataset(key.upper(), dict_with_structure)
        print('.... Merging tables of', target.upper())
        final_dataset_target = merge_all_tables_in_dataset(target.upper(), dict_with_structure)
        final_dataset_individual_coins = pd.merge(final_dataset_key, final_dataset_target, on='last_start_time')
        print(key.upper(), 'and', target.upper(), 'are succesfully merged')
        return final_dataset_individual_coins

In [12]:
def merge_remaining_tables(dict_with_structure):
    print('Now general_info is merged')
    final_dataset_general_info = merge_all_tables_in_dataset('general_info', dict_with_structure)
    return final_dataset_general_info

In [13]:
def final_merger(coinpair):
    print('Merging process started')
    print('')
    dict_with_structure = return_dataset_names(list_not_to_merge)
    print('The structure on Google Bigquery look as follows:')
    for i in dict_with_structure.items():
        print(i)
    print('')
    print('First the tables in', coinpair, 'dataset are merged:')
    final_dataset_per_coinpair = merge_all_tables_in_dataset(coinpair, dict_with_structure)
    print('')
    print(len(final_dataset_per_coinpair))
#     final_dataset_individual_coins = merge_split_coins(coinpair, dict_with_structure)
#     print('')
#     print(len(final_dataset_individual_coins))
#     final_dataset_general_info = merge_remaining_tables(dict_with_structure)
#     print(len(final_dataset_general_info))
#     print('')
#     print('Finally merge all these datasets together:')
#     semi_total_dataset = pd.merge(final_dataset_per_coinpair, final_dataset_individual_coins, on='last_start_time')
#     total_dataset = pd.merge(semi_total_dataset, final_dataset_general_info, on='last_start_time')
#     print('---> Done, all tables are succesfully merged!')
#     return total_dataset
    return final_dataset_per_coinpair

In [14]:
totaal = final_merger("BNBETH")

Merging process started

The structure on Google Bigquery look as follows:
('BTCUSDT', ['technical_analysis_candles', 'ticker_info', 'general_info'])
('BNBETH', ['technical_analysis_candles', 'ticker_info', 'general_info'])
('BNBUSDT', ['technical_analysis_candles', 'ticker_info', 'general_info'])
('ETHUSDT', ['technical_analysis_candles', 'ticker_info', 'general_info'])
('general_info', ['exchange_info'])

First the tables in BNBETH dataset are merged:
ticker_info
general_info
 ---> Done ['technical_analysis_candles', 'ticker_info', 'general_info'] are merged

1224



In [17]:
totaal.tail(100)


,last_start_time,BNBETH__technical_analysis_candles__rsi,BNBETH__technical_analysis_candles__macd,BNBETH__technical_analysis_candles__signal,BNBETH__technical_analysis_candles__macdhist,BNBETH__technical_analysis_candles__sma_5,BNBETH__technical_analysis_candles__sma_10,BNBETH__technical_analysis_candles__sma_21,BNBETH__technical_analysis_candles__sma_50,BNBETH__technical_analysis_candles__sma_100,...,BNBETH__general_info__high_24h,BNBETH__general_info__low_24h,BNBETH__general_info__price_change_24h,BNBETH__general_info__price_change_percentage_24h,BNBETH__general_info__market_cap_change_24h,BNBETH__general_info__market_cap_change_percentage_24h,BNBETH__general_info__circulating_supply,BNBETH__general_info__total_supply,BNBETH__general_info__ath,BNBETH__general_info__ath_change_percentage
1124,1.587464e+12,60.033992,0.000021,0.000008,-0.000014,0.088158,0.088143,0.088090,0.088132,0.088164,...,11.445932,11.163486,0.130553,1.16136,15148156.0,1.21937,1.106083e+08,None,7747.0,-99.85328
1125,1.587464e+12,60.033992,0.000023,0.000011,-0.000012,0.088170,0.088146,0.088105,0.088131,0.088161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
1126,1.587464e+12,67.362800,0.000029,0.000014,-0.000015,0.088188,0.088157,0.088114,0.088131,0.088159,...,11.445932,11.163486,0.113015,1.00599,15148156.0,1.21937,1.106083e+08,None,7747.0,-99.85328
1127,1.587464e+12,69.569657,0.000035,0.000018,-0.000016,0.088205,0.088170,0.088128,0.088132,0.088157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
1128,1.587464e+12,69.569657,0.000039,0.000023,-0.000017,0.088221,0.088183,0.088139,0.088134,0.088156,...,11.445932,11.163486,0.112014,0.99708,15148156.0,1.21937,1.106083e+08,None,7747.0,-99.85328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,1.587397e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.354785,11.133665,0.197028,1.76736,21335941.0,1.73102,1.105975e+08,None,7747.0,-99.85367
1220,1.587394e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.339390,11.107638,0.217653,1.95542,21894816.0,1.77750,1.105971e+08,None,7747.0,-99.85352
1221,1.587467e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.445932,11.163486,0.154764,1.38116,18346841.0,1.48176,1.106089e+08,None,7747.0,-99.85336
1222,1.587467e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.445932,11.163486,0.154764,1.38116,18346841.0,1.48176,1.106089e+08,None,7747.0,-99.85336


In [18]:
totaal = totaal.sort_values('last_start_time', ascending=True)

In [20]:
totaal.to_csv(r'totale_dataset_new_coins.csv', index = False)

In [ ]:
for i in totaal.columns:
    print(i)

In [ ]:
totaal